In [ ]:
import os, json, random
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

def car_inspector(path_to_file):
    img = cv2.imread(path_to_file)
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(path_to_file, out.get_image()[:, :, ::-1])

    bound_area = (outputs["instances"].pred_boxes.area().tolist())
    bound_classes = (outputs["instances"].pred_classes.tolist())
    num_cars = sum([bound_class==2 for bound_class in bound_classes])
    little_car = True
    if num_cars > 0:

        biggest_area = sorted(bound_area)[-1]
        big_area = biggest_area*0.5 # to take bounding box with area 50% and bigger of the biggest area detected

        detected_car = []
        for j in range(len(bound_classes)):
            if str(bound_classes[j]) == '2':
                if bound_area[j] > big_area:
                    little_car = False
                    index_area = (j, bound_area[j])
                    detected_car.append(index_area)
                
        if little_car == False:
            biggest_car = max(detected_car, key=lambda index_area:index_area[1])
            print(biggest_car)
            mask = outputs["instances"].pred_boxes[biggest_car[0]]
            mask = mask.tensor.tolist()

            x = int(float(mask[0][0]))
            y = int(float(mask[0][1]))
            w = int(float(mask[0][2]))
            h = int(float(mask[0][3]))

            croped_car = Image.open(path_to_file)
            croped_car = croped_car.crop((x, y, w, h)).save(path_to_file)
        else: 
            print('на фото '+ path_to_file + 'нет автомобиля подходящего размера')
    else:
        print('на фото '+ path_to_file + 'нет автомобиля!'
    

In [ ]:
#директории на локальном компьютере, где проводилась предобработка датасета.
dir_img = 'D:/car_dataset/_learn/'
dir_train = dir_img + 'train/'
dir_test = dir_img + 'test/'

In [ ]:
#Обрабатываем каждое изображение Детектроном2 и вырезаем автомобиль по баундинг-боксу
for car_model in os.listdir(dir_train):
    for filename in os.listdir(dir_train + car_model + '/'):
        path_to_file = dir_train + car_model +'/'+filename
        biggest_car_in_bounds(path_to_file)

In [ ]:
#Приводим все фотки к квадратному виду без искажения изображений. Затем ресайзим до размера 331x331 (с этим разрешением
#работает используемая нами нейросеть NasNetLarge)

for car_model in tqdm(os.listdir(dir_train)):
    for filename in os.listdir(dir_train + car_model + '/'):
        img  = Image.open(dir_train + car_model +'/'+filename)
        max_size = max(img.size)
        
        new_size = (max_size, max_size)
        new_im = Image.new("RGB", new_size)
        new_im.paste(img, (round((new_size[0]-img.size[0])/2),round((new_size[1]-img.size[1])/2)))
        new_im = new_im.resize((331,331))
        new_im.save(dir_train+car_model+'/'+filename)

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [12:01<00:00,  8.11s/it]


In [ ]:
#Переносим ~30% картинок по каждому классу в валидационную папку

for class_dir in tqdm(os.listdir(dir_train)):
    f = os.listdir(dir_train + class_dir)
    count_files = len(f)
    i = 0
    os.mkdir(dir_train + class_dir)
    for filename in os.listdir(dir_train + class_dir):
        
        os.rename(dir_train + class_dir + '/' + filename, dir_test + class_dir+'/'+ filename)
        i += 1
        if i >= round(count_files*0.3):
            break

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:01<00:00, 45.25it/s]


In [ ]:
#Начало работы на Google Colab. Переносим наш датасет в локальное окружение сервера.
!mkdir '/content/Cars/'
!cp "/content/drive/MyDrive/datasets/Cars/car/crop.zip" '/content/Cars/'


In [ ]:
!unzip "/content/Cars/crop.zip" -d "/content/Cars"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_dir= '/content/Cars/crop'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

In [ ]:
#Загружаем модель NASNetLarge, присоединяем к ней новую "голову", и получаем нашу модель, которую нужно дообучить.

model_NASNetLarge = keras.applications.NASNetLarge(
    include_top=False,
    weights="imagenet",
    input_shape=(331,331,3))

x = model_NASNetLarge.output
x = layers.GlobalAveragePooling2D()(x)
          
output = layers.Dense(89, activation='softmax')(x)

model_with_NASNetLarge=keras.Model(inputs=model_NASNetLarge.inputs, outputs=output)
#model_with_NASNetLarge.summary()

343613440/343610240 [==============================] - 7s 0us/step


In [ ]:
#Замораживаем все слои, кроме последнего. Последний будем учить. 
for layer in model_with_NASNetLarge.layers[:-1]:
    layer.trainable = False

In [ ]:
# Компиляция модели. В качестве метрик задаём accuracy и Top4_Accuracy. Последняя метрика для нас важна, посльку telegram-bot будет выходить
# 4 первых предсказания модели автомобиля.

model_with_NASNetLarge.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy',keras.metrics.TopKCategoricalAccuracy(
    k=4, name="top_4_accuracy", dtype=None)]
)

In [ ]:
#Создание генераторов изображений, которые будут брать файлы из нужных папок и скармливать обучающейся модели.
#Делаем небольшую аугументацию, чтобы предотвратить достаточно быстрое переобучение модели. 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, 
        rotation_range=15, 
        brightness_range=(0.9, 1.1),
        channel_shift_range=50,
        horizontal_flip=True,
        )

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(331, 331),  
        batch_size=64,
        class_mode='categorical'
        )

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(331, 331),
        batch_size=64,
        class_mode='categorical'
        )

Found 13186 images belonging to 89 classes.
Found 5653 images belonging to 89 classes.


In [ ]:
#отдельно создаём объект для колбэка, который будет сохранять модель при улучшении val_top_4_accuracy по результатам каждой эпохи.
#важно объявить объект отдельно, а не в fit_generator, иначе объект будет создаваться каждый раз при запуске обучения, из-за чего
# информация о лучшем val_top_4_accuracy обнулится. 

checkpoint = keras.callbacks.ModelCheckpoint(filepath = '/content/drive/MyDrive/models/car1/', monitor='val_top_4_accuracy', verbose=1, save_best_only=True, save_weights_only=False)

In [ ]:
#Запускаем обучение

history = model_with_NASNetLarge.fit_generator(
      train_generator,
      steps_per_epoch=206, 
      callbacks = checkpoint,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=88, 
      )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1/1 [==============================] - 117s 117s/step - loss: 2.5257 - accuracy: 0.7031 - top_4_accuracy: 0.8750 - val_loss: 3.0228 - val_accuracy: 0.6085 - val_top_4_accuracy: 0.8599


In [ ]:
model_with_NASNetLarge.save('/content/drive/MyDrive/models/car1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/models/car/assets


In [ ]:
model_with_NASNetLarge = keras.models.load_model('/content/drive/MyDrive/models/car')